In [ ]:
import sys
import pandas as pd
sys.path.append('../')
from src.EspnFantasyRoundAnalysis import EspnFantasyRoundAnalysis
from src.utils.io_json import read_json

# Input variables

In [ ]:
week = 17  # Change this to round/week of the fantasy league that you want the analysis for.
scoring_period = 118  # change this to the day of the fantasy league since the start. This is roughly 7*`week`.
# Warning: If you get a matchup period, see message starting with "Processing matchup period:" below other than that specified in the `week` above, 
# reconsider the `scoring_period` variable again.

# Fetch Data

In [ ]:
espn = EspnFantasyRoundAnalysis(round=week, scoring_period=118)

# Stats of the round per Fantasy Team

In [ ]:
espn.get_adv_stats_of_round()

## Ranking index for each statistical category

In [ ]:
espn.compute_stats_ranking_of_round()

# Head to Head scores for all possible match-ups in the specified round

In [ ]:
espn.compute_h2h_score_table()

## Percentage wins from the H2H matchups

In [ ]:
espn.win_ratio_in_round()